## Εργασία 3 ##

Καλωσήρθατε στην τρίτη σας εργασία. Η εργασία αυτή έχει σκοπό να σας βοηθήσει να εμπεδώσετε τα σύνολα μοντέλων.

Στην εργασία αυτή θα πρέπει να συμπληρώσετε κώδικα Python 3 στα σημεία που αναφέρουν # YOUR CODE HERE. Μην τροποποιείτε τον κώδικα που βρίσκεται εκτός αυτών των περιοχών.

Πρωτού παραδόσετε την εργασία σας σιγουρευτείτε ότι ο κώδικας σε όλα τα κελιά τρέχει σωστά. Για το σκοπό αυτό επιλέξτε από το μενού Χρόνος εκτέλεσης (runtime) -> Επανεκίνηση περιόδου λειτουργίας και εκτέλεση όλων.

Συμπληρώστε το όνομα (NAME) και το AEM σας παρακάτω:

In [1]:
NAME = "Iliana Kogia"
AEM = "10090"

**1** Διαβάστε το διαθέσιμο από την sklearn σύνολο δεδομένων breast cancer, χωρίστε το σε δεδομένα εκπαίδευσης (X_train, y_train) και ελέγχου (X_test, y_test) σε ποσοστό 70%/30% αντίστοιχα με τη συνάρτηση train_test_split (τιμή για random_state βάλτε 0). Το σύνολο αφορά τη διάγνωση καρκίνου του μαστού με βάση μεταβλητές που υπολογίζονται από μια ψηφιοποιημένη εικόνα δείγματος μάζας μαστού που λήφθηκε μέσω αναρρόφησης λεπτής βελόνας (FNA). (2 μονάδες)

In [2]:
# YOUR CODE HERE
# raise NotImplementedError()
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

breast_cancer = load_breast_cancer()
X = breast_cancer.data
y = breast_cancer.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

In [3]:
"""Τεστ ορθής ανάγνωσης και διαχωρισμού του συνόλου δεδομένων"""
assert round(X_train[0][8], 5) == 0.1779
assert round(X_test[0][8], 5) == 0.2116

**2** Υλοποιήστε μια ντετερμινιστική εκδοχή της μεθόδου των τυχαίων υποχώρων, η οποία χτίζει τόσα μοντέλα όσες και οι μεταβλητές εισόδου, κάθε ένα από τα οποία αγνοεί και μία διαφορετική μεταβλητή εισόδου. Π.χ. το πρώτο μοντέλο αγνοεί την πρώτη, το δεύτερο αγνοεί τη δεύτερη κτλ. Χρησιμοποιήστε τη συνάρτηση clone από το sklearn.base για να δημιουργήστε αντίγραφο του βασικού μοντέλου σε κάθε επανάληψη. (4 μονάδες)

In [4]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.base import clone

class RandomSubspaceDet:
    def __init__(self, estimator=DecisionTreeClassifier()):
        # YOUR CODE HERE
        # raise NotImplementedError()
        
        self.estimator = estimator
        self.models = []
        self.num_features = 0

    def fit(self, X_train, y_train):
        # YOUR CODE HERE
        # raise NotImplementedError()
        
        self.num_features = X_train.shape[1]
        self.models = []
        
        for i in range(self.num_features):
            
            model = clone(self.estimator) #clone initial model
            X_train_subspace = np.delete(X_train, i, axis = 1) #delete column i of X_train
            model.fit(X_train_subspace, y_train)
            
            self.models.append(model)

    def predict(self, X):
        # YOUR CODE HERE
        # raise NotImplementedError()
        
        predict_array = np.zeros((X.shape[0], self.num_features))
        
        for i, model in enumerate(self.models):
            X_subspace = np.delete(X, i, axis = 1)
            predict_array[:,i] = model.predict(X_subspace)
            
        final_predictions = np.apply_along_axis(lambda x: np.bincount(x.astype(int)).argmax(), axis=1, arr=predict_array)
        return final_predictions

In [5]:
"""Τεστ ορθής υλοποίησης RandomSubspaceDet"""
from sklearn.metrics import accuracy_score

rs = RandomSubspaceDet(estimator=DecisionTreeClassifier(random_state=1))
rs.fit(X_train, y_train)
assert round(accuracy_score(rs.predict(X_test), y_test), 4) == 0.9006

**3** Υλοποιήστε τη μέθοδο AdaBoost όπως παρουσιάστηκε στο μάθημα. Χρησιμοποιήστε τη συνάρτηση clone από το sklearn.base για να δημιουργήστε αντίγραφο του βασικού μοντέλου σε κάθε επανάληψη. Χρησιμοποιήστε την παράμετρο sample_weight της fit του βασικού μοντέλου για να ορίσετε τα βάρη των παραδειγμάτων εκπαίδευσης. (4 μονάδες)

In [39]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.base import clone

class AdaBoost:
    def __init__(self, n_estimators=20, estimator=DecisionTreeClassifier(max_depth=1)):
        # YOUR CODE HERE
        # raise NotImplementedError()
        
        self.n_estimators = n_estimators
        self.base_estimator = estimator
        self.estimators_ = []
        self.estimator_weights_ = []

    def fit(self, X_train, y_train):
        # YOUR CODE HERE
        # raise NotImplementedError()
        
        n_samples, n_features = X_train.shape
        self.sample_weights_ = np.ones(n_samples) / n_samples
        
        # 0,1 labels to -1,1
        y_train = np.where(y_train == 0, -1, 1)
        
        for i in range(self.n_estimators):
            
            estimator = clone(self.base_estimator)
            estimator.fit(X_train, y_train, sample_weight=self.sample_weights_)
            y_pred = estimator.predict(X_train)
            
            incorrect = (y_pred != y_train)
            error = np.dot(self.sample_weights_, incorrect) / np.sum(self.sample_weights_)
            
            if error > 0.5:
                break
            
            alpha = np.log((1 - error) / error) 
            
            self.estimators_.append(estimator)
            self.estimator_weights_.append(alpha)
            
            self.sample_weights_ *= np.exp(alpha * incorrect * (self.sample_weights_ > 0))
            self.sample_weights_ /= np.sum(self.sample_weights_)

    def predict(self, X):
        # YOUR CODE HERE
        # raise NotImplementedError()
        
        predictions = np.array([estimator.predict(X) for estimator in self.estimators_])
        weighted_predictions = np.dot(self.estimator_weights_, predictions)
        
        return np.where(np.sign(weighted_predictions) == -1, 0, 1)

In [40]:
"""Τεστ ορθής υλοποίησης AdaBoost"""
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score

ab = AdaBoost(n_estimators=20, estimator=DecisionTreeClassifier(max_depth=1, random_state=1))
ab.fit(X_train, y_train)
assert round(accuracy_score(ab.predict(X_test), y_test), 4) == 0.9591


In [41]:
# Ίδιο αποτέλεσμα και με τη κλάση της sklearn
ab = AdaBoostClassifier(n_estimators=20, algorithm="SAMME", estimator=DecisionTreeClassifier(max_depth=1, random_state=1))
ab.fit(X_train, y_train)
assert round(accuracy_score(ab.predict(X_test), y_test), 4) == 0.9591